In [1]:
import pandas as pd
import numpy as np
import os


import pandas_datareader as pdr  # to install: !pip install pandas_datareader
from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt 
from scipy import stats
from sklearn.linear_model import LinearRegression

from statsmodels.formula.api import ols as sm_ols
from statsmodels.iolib.summary2 import summary_col # nicer tables
import statsmodels.api as sm

# importing required modules
from zipfile import ZipFile
import zipfile
import time

In [2]:
#Compustat data - roughly 1min to load 
fundamentals = pd.concat((chunk for chunk in pd.read_csv('fundamentals.csv',chunksize=100000, low_memory=False)))
fundamentals.shape #~500k rows, 659 columns 

(501147, 659)

In [3]:
fundamentals['cusip'] = fundamentals['cusip'].astype(str).str[:-1]

In [4]:
fundamentals['cusip']

0         00036110
1         00036110
2         00036110
3         00036110
4         00036110
            ...   
501142    21077C10
501143    21077C10
501144    21077C10
501145    21077C10
501146    21077C10
Name: cusip, Length: 501147, dtype: object

In [5]:
#CRSP data - roughly 10 seconds to load 
returns = pd.concat((chunk for chunk in pd.read_csv('returns.csv',chunksize=100000, low_memory=False)))
returns.shape #~5.7 million rows, 16 columns

(5711711, 16)

In [6]:
#cleaning up returns dataset - roughly 10 sec to load

returns['date'] = pd.to_datetime(returns['date'], format='%Y%m%d', errors='coerce') #change dates format 
returns.sort_values(by='date') # This now sorts in date order
returns #returns in number format, not percentages 
returns[['prc','vol','ret','shrout','sprtrn']] = returns[['prc','vol','ret','shrout','sprtrn']].apply(pd.to_numeric, errors='coerce') #had to change the num to to numeric values 
#returns['ticker'].nunique() #have stock return data on 9,670 firms 
returns.head()

,Unnamed: 0,permno,date,siccd,ncusip,ticker,comnam,shrcls,naics,prc,vol,ret,shrout,cfacpr,cfacshr,sprtrn
0,0,10026,2018-01-02,2052,46603210,JJSF,J & J SNACK FOODS CORP,NaN,311821.0,149.17999,190618.0,-0.017454,18668.0,1.0,1.0,0.008303
1,1,10026,2018-01-03,2052,46603210,JJSF,J & J SNACK FOODS CORP,NaN,311821.0,147.69000,63693.0,-0.009988,18668.0,1.0,1.0,0.006399
2,2,10026,2018-01-04,2052,46603210,JJSF,J & J SNACK FOODS CORP,NaN,311821.0,149.73000,127552.0,0.013813,18668.0,1.0,1.0,0.004029
3,3,10026,2018-01-05,2052,46603210,JJSF,J & J SNACK FOODS CORP,NaN,311821.0,148.30000,44647.0,-0.009550,18668.0,1.0,1.0,0.007034
4,4,10026,2018-01-08,2052,46603210,JJSF,J & J SNACK FOODS CORP,NaN,311821.0,148.41000,55014.0,0.000742,18668.0,1.0,1.0,0.001662


In [7]:
fundamentals.sort_values(by='fyearq').reset_index() # This now sorts in date order
fundamentals.head()

,Unnamed: 0,gvkey,datadate,fyearq,fqtr,fyr,indfmt,consol,popsrc,datafmt,...,prirow,priusa,sic,spcindcd,spcseccd,spcsrc,state,stko,weburl,ipodate
0,0,1004,2010-02-28,2009.0,3.0,5.0,INDL,C,D,STD,...,NaN,1.0,5080.0,110.0,925.0,B,IL,0.0,www.aarcorp.com,1988-01-01
1,1,1004,2010-05-31,2009.0,4.0,5.0,INDL,C,D,STD,...,NaN,1.0,5080.0,110.0,925.0,B,IL,0.0,www.aarcorp.com,1988-01-01
2,2,1004,2010-08-31,2010.0,1.0,5.0,INDL,C,D,STD,...,NaN,1.0,5080.0,110.0,925.0,B,IL,0.0,www.aarcorp.com,1988-01-01
3,3,1004,2010-11-30,2010.0,2.0,5.0,INDL,C,D,STD,...,NaN,1.0,5080.0,110.0,925.0,B,IL,0.0,www.aarcorp.com,1988-01-01
4,4,1004,2011-02-28,2010.0,3.0,5.0,INDL,C,D,STD,...,NaN,1.0,5080.0,110.0,925.0,B,IL,0.0,www.aarcorp.com,1988-01-01


In [8]:
#group returns by quarter
#group return dates by 3/31, 6/30, 9/30, 12/31 


returns['Gross_Ret'] = 1 + returns['ret']

q2_2018 = ((returns[(returns['date'] >= '2018-03-31') & (returns['date'] <= '2018-06-30')]).groupby(['ncusip','ticker'])['Gross_Ret'].prod()-1).reset_index() #Gross Returns 
q3_2018 = ((returns[(returns['date'] >= '2018-06-30') & (returns['date'] <= '2018-09-30')]).groupby(['ncusip','ticker'])['Gross_Ret'].prod()-1).reset_index() #Gross Returns 
q4_2018 = ((returns[(returns['date'] >= '2018-09-30') & (returns['date'] <= '2018-12-31')]).groupby(['ncusip','ticker'])['Gross_Ret'].prod()-1).reset_index() #Gross Returns 

q1_2019 = ((returns[(returns['date'] >= '2018-12-31') & (returns['date'] <= '2019-03-31')]).groupby(['ncusip','ticker'])['Gross_Ret'].prod()-1).reset_index() #Gross Returns 
q2_2019 = ((returns[(returns['date'] >= '2019-03-31') & (returns['date'] <= '2019-06-30')]).groupby(['ncusip','ticker'])['Gross_Ret'].prod()-1).reset_index() #Gross Returns 
q3_2019 = ((returns[(returns['date'] >= '2019-06-30') & (returns['date'] <= '2019-09-30')]).groupby(['ncusip','ticker'])['Gross_Ret'].prod()-1).reset_index() #Gross Returns 
q4_2019 = ((returns[(returns['date'] >= '2019-09-30') & (returns['date'] <= '2019-12-31')]).groupby(['ncusip','ticker'])['Gross_Ret'].prod()-1).reset_index() #Gross Returns 

q1_2020 = ((returns[(returns['date'] >= '2019-12-31') & (returns['date'] <= '2020-03-31')]).groupby(['ncusip','ticker'])['Gross_Ret'].prod()-1).reset_index() #Gross Returns 
q2_2020 = ((returns[(returns['date'] >= '2020-03-31') & (returns['date'] <= '2020-06-30')]).groupby(['ncusip','ticker'])['Gross_Ret'].prod()-1).reset_index() #Gross Returns 
q3_2020 = ((returns[(returns['date'] >= '2020-06-30') & (returns['date'] <= '2020-09-30')]).groupby(['ncusip','ticker'])['Gross_Ret'].prod()-1).reset_index() #Gross Returns 
q4_2020 = ((returns[(returns['date'] >= '2020-09-30') & (returns['date'] <= '2020-12-31')]).groupby(['ncusip','ticker'])['Gross_Ret'].prod()-1).reset_index() #Gross Returns 

In [9]:
q1_2019

,ncusip,ticker,Gross_Ret
0,00030710,AAC,0.304962
1,00036020,AAON,0.325110
2,00036110,AIR,-0.116952
3,00037520,ABB,-0.003171
4,00081T10,ACCO,0.280592
...,...,...,...
7688,Y8564W10,TK,0.184951
7689,Y8565J10,TOO,-0.024388
7690,Y8565N10,TNK,0.032023
7691,Y8897Y80,TOPS,-0.029583


In [10]:
# Merge = pd.merge(fundamentals.loc[fundamentals['datadate']]=='2020-06-30', q3_2020a,left_on='cusip', right_on='ncusip', how='right',indicator=True,validate='many_to_one')
# Merge.tail(20)

q1_2018_acct = fundamentals[fundamentals['datadate'] == '2018-03-31']
q2_2018_acct = fundamentals[fundamentals['datadate'] == '2018-06-30']
q3_2018_acct = fundamentals[fundamentals['datadate'] == '2018-09-30']
q4_2018_acct = fundamentals[fundamentals['datadate'] == '2018-12-31']

q1_2019_acct = fundamentals[fundamentals['datadate'] == '2019-03-31']
q2_2019_acct = fundamentals[fundamentals['datadate'] == '2019-06-30']
q3_2019_acct = fundamentals[fundamentals['datadate'] == '2019-09-30']
q4_2019_acct = fundamentals[fundamentals['datadate'] == '2019-12-31']

q1_2020_acct = fundamentals[fundamentals['datadate'] == '2020-03-31']
q2_2020_acct = fundamentals[fundamentals['datadate'] == '2020-06-30']
q3_2020_acct = fundamentals[fundamentals['datadate'] == '2020-09-30']
q4_2020_acct = fundamentals[fundamentals['datadate'] == '2020-12-31']

q1_2018_acct


#df[(df['date'] > '2013-01-01') & (df['date'] < '2013-02-01')]

# Merge = pd.merge(q2_2020_acct, q3_2020,left_on='cusip', right_on='ncusip',how='left',indicator=True)
# Merge

,Unnamed: 0,gvkey,datadate,fyearq,fqtr,fyr,indfmt,consol,popsrc,datafmt,...,prirow,priusa,sic,spcindcd,spcseccd,spcsrc,state,stko,weburl,ipodate
80,80,1045,2018-03-31,2018.0,1.0,12.0,INDL,C,D,STD,...,NaN,4.0,4512.0,605.0,600.0,C,TX,0.0,www.aa.com,NaN
124,124,1050,2018-03-31,2018.0,1.0,12.0,INDL,C,D,STD,...,NaN,1.0,3564.0,345.0,925.0,C,TX,0.0,www.cecoenviro.com,NaN
211,211,1072,2018-03-31,2017.0,4.0,3.0,INDL,C,D,STD,...,NaN,1.0,3670.0,220.0,925.0,B-,SC,0.0,www.avx.com,1973-03-01
251,251,1075,2018-03-31,2018.0,1.0,12.0,INDL,C,D,STD,...,NaN,1.0,4911.0,705.0,700.0,A,AZ,0.0,www.pinnaclewest.com,NaN
295,295,1076,2018-03-31,2018.0,1.0,12.0,INDL,C,D,STD,...,NaN,1.0,6141.0,175.0,976.0,B,UT,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501048,501048,329141,2018-03-31,2018.0,3.0,6.0,INDL,C,D,STD,...,01W,90.0,7372.0,NaN,NaN,NaN,NaN,0.0,www.renalytixai.com,2020-07-17
501064,501064,330227,2018-03-31,2018.0,2.0,9.0,INDL,C,D,STD,...,01W,1.0,4412.0,NaN,NaN,NaN,NaN,0.0,www.castormaritime.com,2019-02-11
501102,501102,335466,2018-03-31,2018.0,1.0,12.0,INDL,C,D,STD,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,3.0,www.hermitage-offshore.com,NaN
501123,501123,345920,2018-03-31,2018.0,1.0,12.0,INDL,C,D,STD,...,NaN,1.0,3524.0,NaN,NaN,NaN,PA,0.0,www.hydrofarm.com,2020-12-10


In [19]:
Merge_q1_2018 = pd.merge(q1_2018_acct, q2_2018, left_on='cusip', right_on='ncusip',how='left',indicator=True)
Merge_q2_2018 = pd.merge(q2_2018_acct, q3_2018, left_on='cusip', right_on='ncusip',how='left',indicator=True)
Merge_q3_2018 = pd.merge(q3_2018_acct, q4_2018, left_on='cusip', right_on='ncusip',how='left',indicator=True)
Merge_q4_2018 = pd.merge(q4_2018_acct, q1_2019, left_on='cusip', right_on='ncusip',how='left',indicator=True)

Merge_q1_2019 = pd.merge(q1_2019_acct, q2_2019,left_on='cusip', right_on='ncusip',how='left',indicator=True)
Merge_q2_2019 = pd.merge(q2_2019_acct, q3_2019,left_on='cusip', right_on='ncusip',how='left',indicator=True)
Merge_q3_2019 = pd.merge(q3_2019_acct, q4_2019,left_on='cusip', right_on='ncusip',how='left',indicator=True)
Merge_q4_2019 = pd.merge(q4_2019_acct, q1_2020,left_on='cusip', right_on='ncusip',how='left',indicator=True)

Merge_q1_2020 = pd.merge(q1_2020_acct, q2_2020,left_on='cusip', right_on='ncusip',how='left',indicator=True)
Merge_q2_2020 = pd.merge(q2_2020_acct, q3_2020,left_on='cusip', right_on='ncusip',how='left',indicator=True)
Merge_q3_2020 = pd.merge(q3_2020_acct, q4_2020,left_on='cusip', right_on='ncusip',how='left',indicator=True)





In [33]:
frames = [Merge_q1_2018, Merge_q2_2018, Merge_q3_2018, Merge_q4_2018, 
          Merge_q1_2019, Merge_q2_2019, Merge_q3_2019, Merge_q4_2019, 
          Merge_q1_2020, Merge_q2_2020, Merge_q3_2020]

Final = pd.concat(frames)
#Final.sort_values(by='datadate',ascending=True).reset_index() # This now sorts in date order

Final.tail(20)
Final.iloc[350:370]

,Unnamed: 0,gvkey,datadate,fyearq,fqtr,fyr,indfmt,consol,popsrc,datafmt,...,spcseccd,spcsrc,state,stko,weburl,ipodate,ncusip,ticker,Gross_Ret,_merge
350,21064,5530,2018-03-31,2017.0,4.0,3.0,INDL,C,D,STD,...,970.0,B+,MN,0.0,www.hawkinsinc.com,NaN,42026110,HWKN,0.005691,both
351,21108,5539,2018-03-31,2018.0,1.0,12.0,INDL,C,D,STD,...,978.0,B+,PA,0.0,www.hcsgcorp.com,NaN,42190610,HCSG,-0.001463,both
352,21152,5543,2018-03-31,2018.0,1.0,12.0,INDL,C,D,STD,...,800.0,B+,OH,0.0,www.welltower.com,NaN,95040Q10,HCN,0.169741,both
353,21196,5559,2018-03-31,2018.0,1.0,12.0,INDL,C,D,STD,...,925.0,B-,TX,0.0,www.strlco.com,NaN,85924110,STRL,0.137001,both
354,21240,5560,2018-03-31,2018.0,1.0,12.0,INDL,C,D,STD,...,970.0,B-,ID,0.0,www.hecla-mining.com,NaN,42270410,HL,-0.051173,both
355,21331,5568,2018-03-31,2018.0,1.0,12.0,INDL,C,D,STD,...,978.0,B+,PA,0.0,www.kraftheinzcompany.com,NaN,50075410,KHC,0.019364,both
356,21419,5581,2018-03-31,2018.0,2.0,9.0,INDL,C,D,STD,...,935.0,B-,OK,0.0,www.hpinc.com,NaN,42345210,HP,-0.032921,both
357,21464,5583,2018-03-31,2018.0,1.0,12.0,INDL,C,D,STD,...,935.0,C,BC,0.0,www.hemisphereenergy.ca,NaN,NaN,NaN,NaN,left_only
358,21512,5597,2018-03-31,2018.0,1.0,12.0,INDL,C,D,STD,...,978.0,A,PA,0.0,www.thehersheycompany.com,NaN,42786610,HSY,-0.052890,both
359,21556,5600,2018-03-31,2018.0,1.0,12.0,INDL,C,D,STD,...,976.0,D,FL,3.0,www.hertz.com,1997-02-28,42806J10,HTZ,-0.227205,both


In [35]:
Final.shape

(110262, 663)

In [44]:
Final['ticker'].nunique() #8012 unique tickers 
Final['ncusip'].nunique() #7944 unique ncusip
#ticker and ncusip should match up!!!!

Final['cusip'].nunique() #11,453 unique cusip identifers 
Final['ticker'].value_counts()
Final['ncusip'].value_counts()


G9376R10    14
76169B10    14
15976510    14
34623210    13
63905010    12
            ..
45782C27     1
G3728810     1
31748510     1
82835W10     1
G9444H10     1
Name: ncusip, Length: 7944, dtype: int64